# 0.1 Импорт

In [22]:
import nltk
nltk.download('punkt')

import os
from nltk.tokenize import sent_tokenize
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from nltk.tokenize import sent_tokenize
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import BigBirdPegasusForConditionalGeneration, AutoTokenizer
import pandas as pd

from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import meteor_score
from nltk.translate.nist_score import sentence_nist
import pyrouge 
import json

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mitya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# 1.1 Перевод

https://huggingface.co/Helsinki-NLP/opus-mt-ru-en 

### Токенизация текста из файла (предложения)

читаем текст предполагаемой лекции/учебные материалы из файла

In [2]:
with open("./data/test_text.txt", encoding='utf-8') as file:
    text_test = file.read()

In [3]:
text_test = text_test.replace("\n", " ")
text_tokenized = sent_tokenize(text_test, language="russian")

### Переводим текст по 1 предложению

In [35]:
def load_or_download_model(local_path, download_name):
    tokenizer_path = local_path + '-tokenizer'
    model_path = local_path + '-model'
    
    if os.path.exists(tokenizer_path) and os.path.exists(model_path):
        tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
        model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
        
    else:
        tokenizer = AutoTokenizer.from_pretrained(download_name)
        model = AutoModelForSeq2SeqLM.from_pretrained(download_name)
        tokenizer.save_pretrained(tokenizer_path)
        model.save_pretrained(model_path)
        
    return tokenizer, model

In [36]:
tokenizer, model = load_or_download_model("./data/models/translater_ru_en", "Helsinki-NLP/opus-mt-ru-en")

In [5]:
def translate_sentence(inp_t):
    text_to_translate = "translate Russian to English: " + inp_t.lower()

    input_ids = tokenizer(text_to_translate, return_tensors="pt").input_ids

    outputs = model.generate(input_ids)

    text_translated = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return text_translated[30:]

In [7]:
translated_text_tokenized = []

for curr_sentence in text_tokenized:
    translated_text_tokenized.append(translate_sentence(curr_sentence))

In [8]:
# убираем пустые символы в начале строки
translated_text_tokenized = [each_string.lstrip() for each_string in translated_text_tokenized]

In [9]:
# каждое предл начинается с заглавной
translated_text_tokenized = [each_string.capitalize() for each_string in translated_text_tokenized]

In [10]:
translated_text = ' '.join(translated_text_tokenized)

"This article is an attempt to explain the principles of solid by using the example of the java psevod. The article will be useful for beginners to understand these design principles. First, consider the general concept of what is solid and how each letter of the acronym is deciphered. Solid is the software development principles that will give you a good code that will then be well scaled up and maintained in working order. S is the principle of sole responsibility. Each class should have only one area of responsibility. O – open-closed principle. Classes should be open to expansion but closed to change. L - liskov substitution principle is the principle of setting up a fox barn. It should be possible to frame any sub-type of the parent type instead of the parent type (class), without changing the work of the programme. I - interface segregation principle. This principle means that there is no need to force a customer (class) to implement an interface that is not related to it. D - de

In [12]:
with open('./data/temp_steps/raw_eng.txt', 'w') as file:
    file.write(translated_text)

# 1.2 Summarization(TODO)

https://huggingface.co/google/bigbird-pegasus-large-pubmed

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/bigbird-pegasus-large-pubmed")

In [ ]:

# by default encoder-attention is `block_sparse` with num_random_blocks=3, block_size=64
model = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-pubmed")

# decoder attention type can't be changed & will be "original_full"
# you can change `attention_type` (encoder only) to full attention like this:
model = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-pubmed", attention_type="original_full")

# you can change `block_size` & `num_random_blocks` like this:
model = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-pubmed", block_size=16, num_random_blocks=2)


In [ ]:
text = "Solid principles suggest how to design the modules, i.e. the bricks from which the application is built. The purpose of the principles is to design modules that: facilitate change is easily understood again srp: the single responsibility principle a model should be considered to one, and whether one, actor. Old wording: a moduule should have one, and only one, reson to mange. It was often interpreted as follows: the module should have only one duty. "
inputs = tokenizer(text, return_tensors='pt')
prediction = model.generate(**inputs)
prediction = tokenizer.batch_decode(prediction)

Attention type 'block_sparse' is not possible if sequence_length: 97 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 144 with config.block_size = 16, config.num_random_blocks = 2. Changing attention type to 'original_full'...
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (256) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
prediction

['<s> principles suggest how to design modules, i.e., from which the application is built.<n> these principles suggest how to design modules, i.e., that facilitate change is easily understood by one.<n> they suggest how to design modules that are designed by an actor, and are interpreted by an actor.<n> they suggest how to design modules that are not designed by an actor.<n> they suggest how to design modules that are not interpreted by an actor.<n> these principles suggest how to design modules, i.e., that facilitate change is easily understood by one.<n> they suggest how to design modules that are designed by an actor.<n> they suggest how to design modules that are designed by an actor.<n> they suggest how to design modules, i.e., that facilitate change is easily understood by an actor.<n> they suggest how to design modules that are designed by an actor.<n> they suggest how to design modules that are not designed by an actor.<n> they suggest how to design modules that are designed by

# 1.3 Модель для генерации вопрос + ответ

https://huggingface.co/potsawee/t5-large-generation-squad-QuestionAnswer?text=A+recurrent+neural+network+%28RNN%29+is+a+class+of+artificial+neural+networks+where+connections+between+nodes+can+create+a+cycle%2C+allowing+output+from+some+nodes+to+affect+subsequent+input+to+the+same+nodes. seems so


In [37]:
tokenizer, model = load_or_download_model("./data/models/t5_qa_gen", "potsawee/t5-large-generation-squad-QuestionAnswer")

context as an example

In [47]:
def generate_q_and_a(context):
    inputs = tokenizer_qa(context, return_tensors="pt")

    outputs = model_qa.generate(**inputs, max_length=100)

    question_answer = tokenizer_qa.decode(outputs[0], skip_special_tokens=False)

    question_answer = question_answer.replace(tokenizer_qa.pad_token, "").replace(tokenizer_qa.eos_token, "")

    question, answer = question_answer.split(tokenizer_qa.sep_token)

    return question, answer

In [48]:
def generate_q_and_a_line(context):
    q, a = generate_q_and_a(context)
    return str(context + "  " + q +  "  " + a)

1 и 2 - мало, выходят глупости. Надо подумать, сколько брать и мб как-то по умному делить на части. 
Пока делим по 3 предложения на вызов ф-ии.

In [30]:
final_out  =  " "

for current, next, next_next in zip(translated_text_tokenized[::3], translated_text_tokenized[1::2], translated_text_tokenized[2::3]):
  final_out = final_out + generate_q_and_a((current + next) + next_next)

CONTEXT:  This article is an attempt to explain the principles of solid by using the example of the java psevod.The article will be useful for beginners to understand these design principles.First, consider the general concept of what is solid and how each letter of the acronym is deciphered.
QUESTION:  What is the general concept of solid?
ANSWER:  what is solid


CONTEXT:  Solid is the software development principles that will give you a good code that will then be well scaled up and maintained in working order.Solid is the software development principles that will give you a good code that will then be well scaled up and maintained in working order.Each class should have only one area of responsibility.
QUESTION:  What is the name of the software development principles that will give you a good code?
ANSWER:  Solid


CONTEXT:  O – open-closed principle.Each class should have only one area of responsibility.L - liskov substitution principle is the principle of setting up a fox barn.


In [31]:
final_out

" This article is an attempt to explain the principles of solid by using the example of the java psevod.The article will be useful for beginners to understand these design principles.First, consider the general concept of what is solid and how each letter of the acronym is deciphered.   What is the general concept of solid?   what is solidSolid is the software development principles that will give you a good code that will then be well scaled up and maintained in working order.Solid is the software development principles that will give you a good code that will then be well scaled up and maintained in working order.Each class should have only one area of responsibility.   What is the name of the software development principles that will give you a good code?   SolidO – open-closed principle.Each class should have only one area of responsibility.L - liskov substitution principle is the principle of setting up a fox barn.   What is the principle of setting up a fox barn?   L - liskov sub

# 1.4 Q + A по саммаризации (TODO)

# 1.5 Перевод с английского на русский

In [38]:
# переносы строк тоже сразу заменим на пробелы 
final_out = final_out.replace("\n", " ")

text_tokenized = sent_tokenize(final_out, language="english")

In [40]:
tokenizer, model = load_or_download_model("./data/models/translater_en_ru", "Helsinki-NLP/opus-mt-en-ru")

In [41]:
def translate_sentence_to_rus(inp_t):
  text_to_translate = "translate english to russian: " + inp_t.lower()

  input_ids = tokenizer(text_to_translate, return_tensors="pt").input_ids

  outputs = model.generate(input_ids)

  text_translated = tokenizer.decode(outputs[0], skip_special_tokens=True)

  return text_translated

In [42]:
translate_sentence_to_rus("Hello")

C:\Users\Mitya\anaconda3\envs\q_gen_env\lib\site-packages\transformers\generation\utils.py:1346: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'перевести английский на русский: привет'

In [43]:
translated_text_tokenized = []

for curr_sentence in text_tokenized:
  translated_text_tokenized.append(translate_sentence_to_rus(curr_sentence))

In [44]:
translated_text_tokenized

['Переведите английский на русский: эта статья представляет собой попытку объяснить принципы твердости, используя пример Java psevod. Эта статья будет полезной для начинающих, чтобы понять эти принципы дизайна. Во-первых, рассмотрим общую концепцию того, что является твердым и как расшифровывается каждая буква акронима.',
 'перевести английский на русский: какова общая концепция твердого?',
 'Перевод английского языка на русский язык: что твердо, так это принципы разработки программного обеспечения, которые дадут вам хороший код, который затем будет хорошо расширен и сохранен в рабочем состоянии. Принципы разработки программного обеспечения, которые дадут вам хороший код, затем будут значительно расширены и сохранены в рабочем состоянии. Каждый класс должен иметь только одну область ответственности.',
 'перевести английский на русский: как называется принципы разработки программного обеспечения, которые дадут вам хороший код?',
 'Перевод английского на русский: solido – открытый принци

# 2.1 Тестирование

In [49]:
with open('data/xquad.en.json', 'r', encoding='utf-8') as file:
    xquad_data = json.load(file)

In [ ]:
contexts = []
ref_questions = []
ref_answers = []
cand_questions = []
cand_answers = []
for article in xquad_data['data']:

    for paragraph in article['paragraphs']:
        context = paragraph['context']

        for qa in paragraph['qas']:

            contexts.append(context)

            ref_q = qa['question']
            ref_a = qa['answers'][0]['text']              
            ref_questions.append(ref_q)
            ref_answers.append(ref_a)

            cand_q, cand_a = generate_q_and_a(context)
            cand_questions.append(cand_q)
            cand_answers.append(cand_a)


bleu_questions = sentence_bleu(ref_questions, cand_questions)
bleu_answers = sentence_bleu(ref_answers, cand_answers)

meteor_questions = meteor_score(ref_questions, cand_questions)
meteor_answers = meteor_score(ref_answers, cand_answers)

nist_questions = sentence_nist(ref_questions, cand_questions)
nist_answers = sentence_nist(ref_answers, cand_answers)

rouge = pyrouge.Rouge()
rouge_questions = rouge.get_scores(cand_questions, ref_questions, avg=True)
rouge_answers = rouge.get_scores(cand_answers, ref_answers, avg=True)

In [ ]:
evaluation_scores = {
    'Metric': ['BLEU Questions', 'BLEU Answers', 'METEOR Questions', 'METEOR Answers', 'NIST Questions', 'NIST Answers',
               'ROUGE Questions', 'ROUGE Answers'],
    'Score': [bleu_questions, bleu_answers, meteor_questions, meteor_answers, nist_questions, nist_answers,
              rouge_questions, rouge_answers]
}

df = pd.DataFrame(evaluation_scores)
df